徐州市逐年PM2.5含量数据分析 & 数据可视化 & 使用回归模型预测

判断本地是否已有数据，若无数据，则爬取徐州市历年PM2.5每日数据

In [3]:
import time,requests,re
import os
import pandas as pd
from lxml import etree

def get_data():
    url='http://www.tianqihoubao.com/lishi/xuzhou.html'
    headers={'user-agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.18362", }
    response=requests.get(url, headers=headers)
    html=response.text
    response=etree.HTML(html)
    url_list=response.xpath('//div[@class="box p"]//a/@href')
    for url in url_list:
        url='http://www.tianqihoubao.com'+url
        data=pd.read_html(url, header=0)[0] # , encoding='gbk'
        print(data)
        time.sleep(1)
        data.to_csv("tq_xuzhou2022.csv",mode='a', header=False)


#判断本地是否已有数据，若无则运行爬虫
if(os.path.exists("tq_xuzhou2022.csv")==False):
    get_data()
else:
    print("本地已有数据！")


ValueError: No tables found

In [ ]:
filename = "pm2.5_xuzhou.csv"
df=pd.read_csv(filename,names=['date','quality','AQI','ranking','PM2.5(μg/m3)','Pm10(μg/m3)','So2(μg/m3)','No2(μg/m3)','Co(mg/m3)','O3(μg/m3)'])
df   

#经验证爬取的文件无缺失和异常数据，故无需进行处理

#获取的数据如下，数据有 date、qulaity、AQI、ranking、PM2.5 等字段

In [ ]:
#把时间字符串转为索引
df['time_index']=pd.to_datetime(df['date']) 
df.set_index('time_index',inplace=True)
df
#可以看到时间索引在最左边

In [ ]:
#将每日数据绘制成折线图
plot=df['PM2.5(μg/m3)'].plot(figsize=(25,10),title="Xuzhou Daily PM2.5 (ug/m3) 2013/11 - 2020/05 ",color='darkorange', grid=True,fontsize=15)
fig=plot.get_figure()
fig.savefig("Xuzhou Daily PM2.5.png")

#数据太密集，无法看出数据的趋势，数据需要处理

In [ ]:
import matplotlib.pyplot as plt

#降低数据采样频率
df1=df.resample('M').mean()     #月份采样，取月平均值
 
#绘制折线图：
plt.figure(figsize=(25,10))
data=df1['PM2.5(μg/m3)']
#print(data.index)
#print(data.values)
 
_x=data.index
_y=data.values
_x = [i.strftime("%Y/ %m") for i in _x]
plt.plot(range(len(_x)), _y,'darkorange')
plt.xticks(range(len(_x)), _x, rotation=70)
plt.grid(color='black', linestyle='-', linewidth=0.8,alpha=0.4)
plt.xlabel('date')
plt.ylabel('PM2.5(ug/m3)')
plt.title('Xuzhou Monthly Average PM2.5 (ug/m3) 2013/11 - 2020/05 ')
plt.savefig('Xuzhou Monthly Average PM2.5.png')
plt.show()

In [ ]:
#为了更清楚地看到数据趋势，再次降低数据采样频率
df2=df.resample('Y').mean()   #年份采样，取年平均
 
#绘制折线图：
plt.figure(figsize=(25,10))
data=df2['PM2.5(μg/m3)']
#print(data.index)
#print(data.values)
 
_x=data.index
_y=data.values
_x = [i.strftime("%Y ") for i in _x]
plt.plot(range(len(_x)), _y,'darkorange')
plt.xticks(range(len(_x)), _x, rotation=70)
plt.grid(color='black', linestyle='-', linewidth=0.8,alpha=0.4)
plt.xlabel('date')
plt.ylabel('PM2.5(ug/m3)')
plt.title('Xuzhou Yearly Average PM2.5 (ug/m3) 2013 - 2020 ')
plt.savefig('Xuzhou Yearly Average PM2.5.png')
plt.show()

PM2.5含量逐年下降的主要原因可能是：

（1）政府出台相关政策，控制源头，加强工业粉尘治理

（2）城市汽车限行，控制尾气排放

（3）改善了能源消耗结构

（4）人们对PM2.5的危害有更加深刻的认识和了解


In [ ]:
#为了接下来的数据处理和可视化，需要增加一些字段

df['dayofyear'] = df.index.dayofyear   #添加一年中第几天字段
df['dayofweek'] = df.index.dayofweek   #添加星期几字段（0-6）
df['season'] = df.index.quarter        #添加季节字段（1-4）
df['year']=df.index.year               #增加年字段


In [ ]:
df.nsmallest(10,'PM2.5(μg/m3)')  #查看数据中某日PM2.5含量最低的前10名

In [ ]:
df.nlargest(10,'PM2.5(μg/m3)')     #查看数据中某日PM2.5含量最高的前10名

接着我们要用年份来分组，查看每年PM2.5含量数据的情况并画出相关数据图像

In [ ]:
groupby_year=df['PM2.5(μg/m3)'].groupby(df['year'])  #按年划分数据
groupby_year.describe()  

#可以看到每年的PM2.5的 总和，平均值，最大最小值等等的数据都已自动算好

In [ ]:
#绘制柱状图查看每年PM2.5最大、最小和平均含量
import numpy as np
year=['2013','2014','2015','2016','2017','2018','2019','2020','2021','2022']
y1=groupby_year.min().tolist()    #最小值list

meanlist=groupby_year.mean().tolist()
y2=[round(i,2) for i in meanlist]    #平均值list只保留小数点后3位

y3=groupby_year.max().tolist()     #最大值list

x = np.arange(len(year))
width=0.3

plt.bar(x,y1,width=width,label='min',color='green')
plt.bar(x+width,y2,width=width,label='mean',color='deepskyblue',tick_label=year)
plt.bar(x+2*width,y3,width=width,label='max',color='darkorange')

#在图中每个柱条显示数据大小
for a,b in zip(x,y1):
    plt.text(a,b+0.1,b,ha='center',va='bottom')
for a,b in zip(x,y2):
    plt.text(a+width,b+0.1,b,ha='center',va='bottom')
for a,b in zip(x,y3):
    plt.text(a+2*width,b+0.1,b,ha='center', va='bottom')


plt.rcParams['figure.figsize']=(25.0,10.0)
plt.xlabel('year')
plt.ylabel('PM2.5(ug/m3)')
plt.xticks()
plt.legend(loc="upper left")
plt.title("Yearly Compare")
plt.savefig('Yearly compare.png')
plt.show()

接着绘制饼图查看每年空气污染程度占比，（这里排除了2013年和2020年的数据，因为数据不完整）

In [ ]:
#按年划分数据，再按空气质量划分数据
df2014=df[df.year==2014]
groupby_quality2014=df2014['AQI'].groupby(df2014['quality'])  
groupby_quality2014.describe()   #2014年数据

In [ ]:
df2015=df[df.year==2015]
groupby_quality2015=df2015['AQI'].groupby(df2015['quality']) 
groupby_quality2015.describe()   #2015年数据

In [ ]:
df2016=df[df.year==2016]
groupby_quality2016=df2016['AQI'].groupby(df2016['quality'])  
groupby_quality2016.describe()   #2016年数据

In [ ]:
df2017=df[df.year==2017]
groupby_quality2017=df2017['AQI'].groupby(df2017['quality'])  
groupby_quality2017.describe()   #2017年数据

In [ ]:
df2018=df[df.year==2018]
groupby_quality2018=df2018['AQI'].groupby(df2018['quality'])  
groupby_quality2018.describe()   #2018年数据

In [ ]:
df2019=df[df.year==2019]
groupby_quality2019=df2019['AQI'].groupby(df2019['quality'])  
groupby_quality2019.describe()    #2019年数据

In [ ]:
#绘制饼图
# labels=['中度污染','优','良','轻度污染']
labels=['严重污染','中度污染','优','良','轻度污染','重度污染']
colors=['red','pink','green','skyblue','orange','black']
labels1=['优','良','轻度污染']
count2014=groupby_quality2014.count().tolist()
count2015=groupby_quality2015.count().tolist()
count2016=groupby_quality2016.count().tolist()
count2017=groupby_quality2017.count().tolist()
count2018=groupby_quality2018.count().tolist()
count2019=groupby_quality2019.count().tolist()

ex=[0.01,0.01,0.01,0.01,0.01,0.01]
ex1=[0.01,0.01,0.01]

pic=plt.figure(figsize=(25,15))

a1=pic.add_subplot(2,3,1)
plt.rcParams['font.sans-serif']='SimHei'
plt.pie(count2014,explode=ex,labels=labels,colors=colors,autopct='%1.1f%%')
plt.title('徐州2014全年空气质量饼图')

a1=pic.add_subplot(2,3,2)
plt.pie(count2015,explode=ex,labels=labels,colors=colors,autopct='%1.1f%%')
plt.title('徐州2015全年空气质量饼图')

a1=pic.add_subplot(2,3,3)
plt.pie(count2016,explode=ex,labels=labels,colors=colors,autopct='%1.1f%%')
plt.title('徐州2016全年空气质量饼图')

a1=pic.add_subplot(2,3,4)
plt.pie(count2017,explode=ex,labels=labels,colors=colors,autopct='%1.1f%%')
plt.title('徐州2017全年空气质量饼图')

a1=pic.add_subplot(2,3,5)
plt.pie(count2018,explode=ex,labels=labels,colors=colors,autopct='%1.1f%%')
plt.title('徐州2018全年空气质量饼图')

a1=pic.add_subplot(2,3,6)
plt.pie(count2019,explode=ex,labels=labels,colors=colors,autopct='%1.1f%%')
plt.title('徐州2019全年空气质量饼图')

plt.savefig('pie.png')



In [ ]:
import seaborn as sns
#画出热力图，查看字段间的相关性
#相关系数的绝对值在【0.5,1】之间是强相关  【0.2,0.5】之间是有一定相关  小于0.1或0.05是无相关
PM_corr = df.corr()
fig=plt.figure(figsize=(15,15))
sns.heatmap(PM_corr,annot=True)
plt.savefig('heatmap.png')

#注重分析PM2.5字段和其他字段的关系

根据热力图可以得出结论：

（1）PM2.5含量与O3含量有一定相关性，与CO，NO2，SO2的含量强相关。主要原因是这些气体都是污染气体。

（2）PM2.5的含量与AQI指数强相关，即PM2.5的含量影响着AQI指数。

（3）PM2.5与年之间是负相关，含量逐年减少

（4）PM2.5与季节的相关性不大


接着使用 普通线性回归模型、岭回归模型和Lasso回归模型  3种不同的回归模型来预测数据。

第一步划分训练集和测试集

In [ ]:
split_percent=0.1
data_train=df.iloc[int(len(df)*split_percent):]  #前80%的数据划为训练集 
data_test=df.iloc[:int(len(df)*split_percent)]   #剩余20%划为测试集   

test_time=data_test['date']
#type(test_time)
data_test    #测试集数据范围是2019年1月17日 —— 2020年5月1日
#data_train

In [ ]:
#剔除字符串字段
data_train=data_train.drop(['date','quality'], axis = 1)
data_test=data_test.drop(['date','quality'], axis = 1)

In [ ]:
y_train=data_train['PM2.5(μg/m3)'].values
x_train=data_train.drop('PM2.5(μg/m3)', axis = 1).values

y_true=data_test['PM2.5(μg/m3)'].values
x_test=data_test.drop('PM2.5(μg/m3)', axis = 1).values

In [ ]:
y_true   #PM2.5含量真实值（2019-01-17 ——  2020-05-01）

使用普通的线性回归模型

In [ ]:
from sklearn.linear_model import LinearRegression
model=LinearRegression()
model.fit(x_train, y_train)
y_pred=model.predict(x_test)

np.around(y_pred,decimals=1)  #控制输出小数点后一位
#PM2.5含量 普通线性回归模型 预测结果

接着使用岭回归模型

In [ ]:
#先选取最佳alpha参数
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV

ridgecv = RidgeCV(alphas=[0.01, 0.1, 0.5, 1, 3, 5, 7, 10, 20, 100])
ridgecv.fit(x_train, y_train)
ridgecv.alpha_  

In [ ]:
reg=Ridge(alpha = 0.1)  #使用上面的alpha参数
reg.fit(x_train, y_train)
y_pred_reg=reg.predict(x_test)

np.around(y_pred_reg,decimals=1)  #控制输出小数点后一位
#PM2.5含量 岭回归模型 预测结果

最后使用Lasso回归模型

In [ ]:
from sklearn.linear_model import Lasso
lasso=Lasso(alpha = 0.01)
lasso.fit(x_train, y_train)
y_pred_lasso=lasso.predict(x_test)
np.around(y_pred_lasso,decimals=1)  #控制输出小数点后一位
#PM2.5含量 Lasso回归模型 预测结果

三种回归模型的评价


In [ ]:
from sklearn.metrics import explained_variance_score,mean_absolute_error,mean_squared_error,median_absolute_error,r2_score

print('线性回归模型评价')
print('数据线性回归模型的平均绝对误差为',mean_absolute_error(y_true,y_pred).round(8))
print('数据线性回归模型的均方误差为',mean_squared_error(y_true,y_pred).round(8))
print('数据线性回归模型的中值绝对误差为',median_absolute_error(y_true,y_pred).round(8))
print('数据线性回归模型的可解释方差值为',explained_variance_score(y_true,y_pred).round(8))
print('数据线性回归模型的R方值为',r2_score(y_true,y_pred).round(8))

In [ ]:
print('岭回归模型评价')
print('数据岭回归模型的平均绝对误差为',mean_absolute_error(y_true,y_pred_reg).round(8))
print('数据岭回归模型的均方误差为',mean_squared_error(y_true,y_pred_reg).round(8))
print('数据岭回归模型的中值绝对误差为',median_absolute_error(y_true,y_pred_reg).round(8))
print('数据岭回归模型的可解释方差值为',explained_variance_score(y_true,y_pred_reg).round(8))
print('数据岭回归模型的R方值为',r2_score(y_true,y_pred_reg).round(8))

In [ ]:
print('Lasso回归模型评价')
print('数据Lasso回归模型的平均绝对误差为',mean_absolute_error(y_true,y_pred_lasso).round(8))
print('数据Lasso回归模型的均方误差为',mean_squared_error(y_true,y_pred_lasso).round(8))
print('数据Lasso回归模型的中值绝对误差为',median_absolute_error(y_true,y_pred_lasso).round(8))
print('数据Lasso回归模型的可解释方差值为',explained_variance_score(y_true,y_pred_lasso).round(8))
print('数据Lasso回归模型的R方值为',r2_score(y_true,y_pred_lasso).round(8))

绘制折线图，看不同模型间的拟合程度

首先将预测的结果转为 DataFrame

In [ ]:
#真实值
time=test_time.values
df_true=pd.DataFrame({'date':time,'PM2.5(μg/m3)':y_true})
df_true['time_index'] = pd.to_datetime(df_true['date']) #把时间字符串转为索引
df_true.set_index('time_index',inplace=True)
#df_true

#普通线性回归模型预测的结果
df_mod1=pd.DataFrame({'date':time,'PM2.5(μg/m3)':y_pred})
df_mod1['time_index'] = pd.to_datetime(df_mod1['date']) #把时间字符串转为索引
df_mod1.set_index('time_index',inplace=True)
#df_mod1

#岭回归模型预测的结果
df_mod2=pd.DataFrame({'date':time,'PM2.5(μg/m3)':y_pred_reg})
df_mod2['time_index'] = pd.to_datetime(df_mod2['date']) #把时间字符串转为索引
df_mod2.set_index('time_index',inplace=True)
#df_mod2

#Lasso回归模型预测的结果
df_mod3=pd.DataFrame({'date':time,'PM2.5(μg/m3)':y_pred_lasso})
df_mod3['time_index'] = pd.to_datetime(df_mod3['date']) #把时间字符串转为索引
df_mod3.set_index('time_index',inplace=True)
#df_mod3

In [ ]:
#绘制图像
df_true['PM2.5(μg/m3)'].plot(figsize=(25,10),color='red', grid=True,fontsize=15,label='True')                       #真实数据
df_mod1['PM2.5(μg/m3)'].plot(figsize=(25,10),color='darkorange', grid=True,fontsize=15,label='LinearRegression')     #普通线性回归模型预测数据
df_mod2['PM2.5(μg/m3)'].plot(figsize=(25,10),color='green', grid=True,fontsize=15,label='Ridge')                    #岭回归模型预测数据
df_mod3['PM2.5(μg/m3)'].plot(figsize=(25,10),color='lightblue', grid=True,fontsize=15,label='Lasso')                 #Lasso回归模型预测数据
plt.xticks()
plt.legend(loc="upper right")
plt.savefig('Model Compare.png')


从图中也可以看到，三种模型预测的结果差别不大